# Look at accelerometer data 

Finding Zero velocity times by rail axis acceleration noise levels, making summary statistics for the noise levels across the whole day files.  Spot check graphs to see what works

In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'

#Standard cycle to make black and white images and dashed and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '-', '-', '-']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

In [2]:
#Extra Headers:
import os as os
import pywt as py
import statistics as st
import os as os
import random
import multiprocessing
from joblib import Parallel, delayed
import platform

from time import time as ti

In [3]:
import CoreFunctions as cf
from skimage.restoration import denoise_wavelet

## Choosing Platform
Working is beinging conducted on several computers, and author needs to be able to run code on all without rewriting..  This segment of determines which computer is being used, and sets the directories accordingly.

In [4]:
HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

In [5]:
if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/image/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
elif Computer == 'PortLap':
    location = 'C:\\users\\dhendrickson\\Desktop\\AccelData\\'

In [6]:
if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    folder = '/sciclone/scr10/dchendrickson01/Recordings2/'
    imageFolder = '/sciclone/scr10/dchendrickson01/Move3Dprint/'
elif Computer == "Desktop":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'Recordings2/'
elif Computer =='PortLap':
    rootfolder = location 
    folder = rootfolder + 'Recordings2\\'

## Global Variables

In [7]:
Saving = False
location = folder
Titles = True
Ledgends = True

f = 0


In [8]:
files = ['230418 recording1.csv','230419 recording1.csv','230420 recording1.csv'] #,'230421 recording1.csv',
         #'230418 recording2.csv','230419 recording2.csv','230420 recording2.csv','230421 recording2.csv']

In [9]:
BeforeTamping = ['221206 recording1.csv','221207 recording1.csv','221208 recording1.csv','221209 recording1.csv',
         '221206 recording2.csv','221207 recording2.csv','221208 recording2.csv','221209 recording2.csv']


## Project Specific Functions

In [10]:
#Smooth = cf.Smoothing(ODataSet[:,3],2) #,50)
def SmoothMoves(file):
    #    if file[-3:] =='csv':
    ODataSet = np.genfromtxt(open(folder+file,'r'), delimiter=',',skip_header=0,missing_values=0,invalid_raise=False)
    SmoothX = denoise_wavelet(ODataSet[:,3], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
    SmoothY = denoise_wavelet(ODataSet[:,4], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
    SmoothZ = denoise_wavelet(ODataSet[:,5], method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
    SmoothX -= np.average(SmoothX)
    SmoothY -= np.average(SmoothY)
    SmoothZ -= np.average(SmoothZ)
    MoveMatrix = np.matrix([SmoothX, SmoothY, SmoothZ])
    return MoveMatrix
    #else:
    #    pass


## Process Files

In [11]:
LoopFiles = 3
loops = int(len(files) / LoopFiles) 
if len(files)%LoopFiles != 0:
    loops += 1


In [12]:
files


['230418 recording1.csv',
 '230419 recording1.csv',
 '230420 recording1.csv',
 '230421 recording1.csv',
 '230418 recording2.csv',
 '230419 recording2.csv',
 '230420 recording2.csv',
 '230421 recording2.csv']

In [13]:

st = ti()

Moves = []

for k in range(loops):
    if k == loops -1:
        tfiles = files[k*LoopFiles:]
    else:
        tfiles = files[k*LoopFiles:(k+1)*LoopFiles]
    #Results = Parallel(n_jobs=LoopFiles)(delayed(DeviationVelocity)(file) for file in tfiles)
    Results = Parallel(n_jobs=LoopFiles)(delayed(SmoothMoves)(file) for file in tfiles)
    #Results =[]
    #for file in tfiles:
    #    Results.append(DeviationVelocity(file))
    #    print(file, (ti()-st)/60.0)
    for result in Results:
        Moves.append(result)
    print(k, (ti()-st)/60.0)
        


0 1.7730672399202982
1 3.4014491995175677
2 4.5378952900568645


## Try LSTM Stuff

https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/

In [14]:
TimeSteps = 250
StepSize = 75
Features = np.shape(Moves[0])[0]

In [15]:
len(Moves)

8

In [16]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps, s_step = 1):
    X, y = list(), list()
    Steps_to_take = int(len(sequences) / s_step)
    for j in range(Steps_to_take):
        i = j * s_step
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [17]:
Sequences = []
Outputs = []
for move in Moves:
    Seq, Out = split_sequences(move.T,TimeSteps,StepSize)
    Sequences.append(Seq)
    Outputs.append(Out)
    

In [18]:
MoveSegments = []
for seq in Sequences:
    for mv in seq:
        MoveSegments.append(mv)
NextDataPoint = []
for out in Outputs:
    for pt in out:
        NextDataPoint.append(np.reshape(pt,(1,3)))

In [19]:
len(MoveSegments)

1713931

In [20]:
from sklearn.model_selection import train_test_split

In [32]:
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Masking
from keras.models import Sequential
import tensorflow as tf


class LSTM_Autoencoder:
  def __init__(self, optimizer='adam', loss='mse'):
    self.optimizer = optimizer
    self.loss = loss
    self.n_features = Features
    self.timesteps = TimeSteps
    
  def build_model(self):
    timesteps = self.timesteps
    n_features = self.n_features
    model = Sequential()
    
    # Padding
    #model.add(Masking(mask_value=0.0, input_shape=(timesteps, n_features)))

    # Encoder
    model.add(LSTM(timesteps, activation='relu', input_shape=(TimeSteps, Features), return_sequences=True))
    model.add(LSTM(35, activation='relu', return_sequences=True))
    model.add(LSTM(6, activation='relu'))
    model.add(RepeatVector(timesteps))
    
    # Decoder
    model.add(LSTM(timesteps, activation='relu', return_sequences=True))
    model.add(LSTM(35, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    
    model.compile(optimizer=self.optimizer, loss=self.loss, metrics=['accuracy'])
    model.summary()
    self.model = model
    
  def simple_model(self):
    
    # define model
    model = Sequential()
    model.add(LSTM(150, input_shape=(self.timesteps, self.n_features)))
    model.add(RepeatVector(self.timesteps))
    model.add(LSTM(150, return_sequences=True))
    model.add(TimeDistributed(Dense(self.n_features, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    self.model = model
    
  def fit(self, X, epochs=3, batch_size=32):
    #self.timesteps = np.shape(X)[0]
    self.build_model()
    
    #input_X = np.expand_dims(X, axis=1)
    self.model.fit(X, X, epochs=epochs, batch_size=batch_size)
    
  def predict(self, X):
    #input_X = np.expand_dims(X, axis=1)
    output_X = self.model.predict(X)
    reconstruction = np.squeeze(output_X)
    return np.linalg.norm(X - reconstruction, axis=-1)
  
  def plot(self, scores, timeseries, threshold=0.95):
    sorted_scores = sorted(scores)
    threshold_score = sorted_scores[round(len(scores) * threshold)]
    
    plt.title("Reconstruction Error")
    plt.plot(scores)
    plt.plot([threshold_score]*len(scores), c='r')
    plt.show()
    
    anomalous = np.where(scores > threshold_score)
    normal = np.where(scores <= threshold_score)
    
    plt.title("Anomalies")
    plt.scatter(normal, timeseries[normal][:,-1], s=3)
    plt.scatter(anomalous, timeseries[anomalous][:,-1], s=5, c='r')
    plt.show()


In [33]:
#split = int(len(Moves)*.9)
#Train_data = Moves[:split]
#Test_data = Moves[split:]
#Train_data = tf.ragged.constant(Train_data)
#Test_data = tf.ragged.constant(Test_data)

In [34]:
lstm_autoencoder2 = LSTM_Autoencoder(optimizer='adam', loss='mse')

In [35]:
lstm_autoencoder2.simple_model()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 150)               92400     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 250, 150)         0         
 or)                                                             
                                                                 
 lstm_5 (LSTM)               (None, 250, 150)          180600    
                                                                 
 time_distributed (TimeDistr  (None, 250, 3)           453       
 ibuted)                                                         
                                                                 
Total params: 273,453
Trainable params: 273,453
Non-trainable params: 0
_________________________________________________________________


In [36]:
Batches = 32
NumbBatches = 100

SamplesPerSet = Batches * NumbBatches


In [37]:
SamplesPerSet

3200

In [38]:
SetsNeeded = int(len(MoveSegments) / SamplesPerSet)
if  int(len(MoveSegments) / SamplesPerSet) != 0:
    SetsNeeded += 1
print(len(MoveSegments), SetsNeeded)

1713931 536


In [39]:
PercentPerSet = 1.0 / float(SetsNeeded)

PercentHoldOutForNext=1.0

In [40]:
st = ti()

for i in range(SetsNeeded-1):
    PercentHoldOutForNext = 1.0 - (SamplesPerSet / len(MoveSegments))
    seq_train, seq_test, out_train, out_test = train_test_split(MoveSegments, NextDataPoint, test_size=PercentHoldOutForNext, shuffle=True, random_state=0)
    seq_train = np.asarray(seq_train)
    lstm_autoencoder2.model.fit(seq_train, seq_train, epochs=2, batch_size=32, verbose=1)
    MoveSegments = seq_test
    NextDataPoint = out_test
    print(str(i+1)+' of ' + str(SetsNeeded), (ti()-st)/60, (((ti()-st)/(i+1) * ( SetsNeeded -1) - (ti()-st) )/60/60)
    
    

Epoch 1/2
100/100 [==============================] - 65s 613ms/step - loss: -7.7297e-04 - accuracy: 0.3398
Epoch 2/2
100/100 [==============================] - 63s 628ms/step - loss: -9.7154e-06 - accuracy: 0.3369
1 of 536 2.1610308369000752 69239.43183207512
Epoch 1/2
100/100 [==============================] - 61s 611ms/step - loss: -8.4382e-05 - accuracy: 0.3504
Epoch 2/2
100/100 [==============================] - 61s 610ms/step - loss: -4.0639e-04 - accuracy: 0.3504
2 of 536 4.219568208853404 67470.89731037617
Epoch 1/2
100/100 [==============================] - 62s 622ms/step - loss: 1.3144e-04 - accuracy: 0.3417
Epoch 2/2
100/100 [==============================] - 62s 624ms/step - loss: 7.5372e-05 - accuracy: 0.3380
3 of 536 6.32037300268809 67248.7698041598
Epoch 1/2
100/100 [==============================] - 59s 589ms/step - loss: 4.0055e-05 - accuracy: 0.3044
Epoch 2/2
100/100 [==============================] - 60s 599ms/step - loss: -8.2304e-05 - accuracy: 0.3044
4 of 536 8.33

Epoch 1/2
100/100 [==============================] - 63s 628ms/step - loss: -0.0034 - accuracy: 0.3048
Epoch 2/2
100/100 [==============================] - 63s 635ms/step - loss: -0.0034 - accuracy: 0.3048
33 of 536 69.49740357001623 63432.175780238525
Epoch 1/2
100/100 [==============================] - 65s 650ms/step - loss: -0.0059 - accuracy: 0.3063
Epoch 2/2
100/100 [==============================] - 62s 619ms/step - loss: -0.0060 - accuracy: 0.3124
34 of 536 71.63746471802394 63335.946834395916
Epoch 1/2
100/100 [==============================] - 64s 636ms/step - loss: -0.0013 - accuracy: 0.3555
Epoch 2/2
100/100 [==============================] - 64s 636ms/step - loss: -0.0013 - accuracy: 0.3401
35 of 536 73.77886140743891 63239.02414066451
Epoch 1/2
100/100 [==============================] - 65s 648ms/step - loss: 6.1383e-04 - accuracy: 0.3425
Epoch 2/2
100/100 [==============================] - 61s 614ms/step - loss: 5.8881e-04 - accuracy: 0.3545
36 of 536 75.9165469288826 631

100/100 [==============================] - 63s 630ms/step - loss: -8.5658e-04 - accuracy: 0.3065
Epoch 2/2
100/100 [==============================] - 63s 631ms/step - loss: -8.5658e-04 - accuracy: 0.3065
97 of 536 205.6511758764585 55716.627872884885
Epoch 1/2
100/100 [==============================] - 61s 608ms/step - loss: -0.0041 - accuracy: 0.3070
Epoch 2/2
100/100 [==============================] - 64s 640ms/step - loss: -0.0041 - accuracy: 0.3070
98 of 536 207.75002915064493 55583.7323099764
Epoch 1/2
100/100 [==============================] - 63s 633ms/step - loss: 0.0054 - accuracy: 0.3016
Epoch 2/2
100/100 [==============================] - 64s 638ms/step - loss: 0.0054 - accuracy: 0.3016
99 of 536 209.8875014901161 55461.18223279173
Epoch 1/2
100/100 [==============================] - 63s 626ms/step - loss: -0.0041 - accuracy: 0.3065
Epoch 2/2
100/100 [==============================] - 61s 613ms/step - loss: -0.0041 - accuracy: 0.3065
100 of 536 211.9707379659017 55324.362641

100/100 [==============================] - 59s 588ms/step - loss: -0.0032 - accuracy: 0.3073
Epoch 2/2
100/100 [==============================] - 61s 611ms/step - loss: -0.0032 - accuracy: 0.3073
161 of 536 340.6482905069987 47479.177898053065
Epoch 1/2
100/100 [==============================] - 64s 636ms/step - loss: -0.0053 - accuracy: 0.3096
Epoch 2/2
100/100 [==============================] - 59s 586ms/step - loss: -0.0053 - accuracy: 0.3096
162 of 536 342.7003009398778 47343.41196069894
Epoch 1/2
100/100 [==============================] - 61s 606ms/step - loss: 0.0015 - accuracy: 0.3017
Epoch 2/2
100/100 [==============================] - 64s 643ms/step - loss: 0.0015 - accuracy: 0.3017
163 of 536 344.8012056628863 47214.49639649625
Epoch 1/2
100/100 [==============================] - 61s 613ms/step - loss: 0.0064 - accuracy: 0.2991
Epoch 2/2
100/100 [==============================] - 64s 636ms/step - loss: 0.0064 - accuracy: 0.2991
164 of 536 346.90226517518363 47085.63673314234


100/100 [==============================] - 62s 623ms/step - loss: 0.0039 - accuracy: 0.3094
Epoch 2/2
100/100 [==============================] - 61s 608ms/step - loss: 0.0039 - accuracy: 0.3094
225 of 536 475.68787913719814 39323.53134792646
Epoch 1/2
100/100 [==============================] - 62s 625ms/step - loss: 0.0032 - accuracy: 0.3035
Epoch 2/2
100/100 [==============================] - 60s 596ms/step - loss: 0.0032 - accuracy: 0.3035
226 of 536 477.73388822078704 39191.08977518251
Epoch 1/2
100/100 [==============================] - 59s 595ms/step - loss: 0.0040 - accuracy: 0.3046
Epoch 2/2
100/100 [==============================] - 59s 587ms/step - loss: 0.0040 - accuracy: 0.3046
227 of 536 479.71693733135857 39053.60794455469
Epoch 1/2
100/100 [==============================] - 59s 591ms/step - loss: 0.0021 - accuracy: 0.3057
Epoch 2/2
100/100 [==============================] - 60s 596ms/step - loss: 0.0021 - accuracy: 0.3057
228 of 536 481.7086748480797 38916.990316534255
Ep

100/100 [==============================] - 63s 628ms/step - loss: 0.0051 - accuracy: 0.3023
Epoch 2/2
100/100 [==============================] - 64s 645ms/step - loss: 0.0051 - accuracy: 0.3023
289 of 536 609.6187844991684 31134.855571085194
Epoch 1/2
100/100 [==============================] - 62s 622ms/step - loss: -0.0042 - accuracy: 0.3059
Epoch 2/2
100/100 [==============================] - 63s 631ms/step - loss: -0.0042 - accuracy: 0.3059
290 of 536 611.7196379582088 31007.857513551047
Epoch 1/2
100/100 [==============================] - 64s 637ms/step - loss: -0.0040 - accuracy: 0.3045
Epoch 2/2
100/100 [==============================] - 61s 609ms/step - loss: -0.0040 - accuracy: 0.3045
291 of 536 613.810446937879 30880.3606329038
Epoch 1/2
100/100 [==============================] - 61s 610ms/step - loss: 0.0123 - accuracy: 0.3006
Epoch 2/2
100/100 [==============================] - 62s 617ms/step - loss: 0.0123 - accuracy: 0.3006
292 of 536 615.8703021287918 30751.332212443216
E

100/100 [==============================] - 63s 633ms/step - loss: 0.0027 - accuracy: 0.3028
Epoch 2/2
100/100 [==============================] - 62s 618ms/step - loss: 0.0027 - accuracy: 0.3028
353 of 536 743.5489225387573 23001.570069741923
Epoch 1/2
100/100 [==============================] - 64s 640ms/step - loss: -0.0026 - accuracy: 0.3050
Epoch 2/2
100/100 [==============================] - 63s 628ms/step - loss: -0.0026 - accuracy: 0.3050
354 of 536 745.6771586974462 22875.858600641383
Epoch 1/2
100/100 [==============================] - 63s 631ms/step - loss: 6.9264e-04 - accuracy: 0.3086
Epoch 2/2
100/100 [==============================] - 62s 624ms/step - loss: 6.9264e-04 - accuracy: 0.3086
355 of 536 747.7774969696999 22749.287232637405
Epoch 1/2
100/100 [==============================] - 60s 605ms/step - loss: 0.0015 - accuracy: 0.3062
Epoch 2/2
100/100 [==============================] - 63s 629ms/step - loss: 0.0015 - accuracy: 0.3062
356 of 536 749.8438883940379 22621.69483

100/100 [==============================] - 63s 631ms/step - loss: 0.0015 - accuracy: 0.3077
Epoch 2/2
100/100 [==============================] - 60s 596ms/step - loss: 0.0015 - accuracy: 0.3077
417 of 536 878.1296163201332 14909.251039459268
Epoch 1/2
100/100 [==============================] - 63s 630ms/step - loss: -0.0040 - accuracy: 0.3024
Epoch 2/2
100/100 [==============================] - 62s 616ms/step - loss: -0.0040 - accuracy: 0.3024
418 of 536 880.2144744237264 14782.54930724026
Epoch 1/2
100/100 [==============================] - 63s 627ms/step - loss: -0.0102 - accuracy: 0.3135
Epoch 2/2
100/100 [==============================] - 63s 628ms/step - loss: -0.0102 - accuracy: 0.3135
419 of 536 882.3184656977653 14656.173082611727
Epoch 1/2
100/100 [==============================] - 63s 631ms/step - loss: -0.0017 - accuracy: 0.3053
Epoch 2/2
100/100 [==============================] - 62s 617ms/step - loss: -0.0017 - accuracy: 0.3053
420 of 536 884.4018450975418 14529.4588836431

100/100 [==============================] - 63s 631ms/step - loss: -6.8694e-04 - accuracy: 0.3053
481 of 536 1012.0665498216947 6817.24661204523
Epoch 1/2
100/100 [==============================] - 62s 622ms/step - loss: 0.0036 - accuracy: 0.3028
Epoch 2/2
100/100 [==============================] - 62s 623ms/step - loss: 0.0036 - accuracy: 0.3028
482 of 536 1014.1489941080412 6690.858508806996
Epoch 1/2
100/100 [==============================] - 61s 608ms/step - loss: 0.0015 - accuracy: 0.3051
Epoch 2/2
100/100 [==============================] - 63s 634ms/step - loss: 0.0015 - accuracy: 0.3051
483 of 536 1016.2242161591847 6564.429718082247
Epoch 1/2
100/100 [==============================] - 62s 624ms/step - loss: -0.0065 - accuracy: 0.3043
Epoch 2/2
100/100 [==============================] - 63s 635ms/step - loss: -0.0065 - accuracy: 0.3043
484 of 536 1018.3293122013409 6438.197716181428
Epoch 1/2
100/100 [==============================] - 64s 639ms/step - loss: -0.0038 - accuracy: 0.

In [41]:

lstm_autoencoder2.model.save("LSTM_sm_FullDays")

INFO:tensorflow:Assets written to: LSTM_sm_FullDays/assets


INFO:tensorflow:Assets written to: LSTM_sm_FullDays/assets


In [ ]:
output_X = lstm_autoencoder2.model.predict(seq_train)

In [ ]:
reconstruction = np.squeeze(output_X)

In [ ]:
scores = np.linalg.norm(seq_train - reconstruction, axis=-1)

In [ ]:
output_X[1]

In [ ]:
maxes = np.max(scores, axis = 0)

In [ ]:
np.shape(maxes)

In [ ]:
lstm_autoencoder2.plot(maxes, seq_train, threshold=0.95)

In [ ]:
LoopFiles = 3
loops = int(len(BeforeTamping) / LoopFiles) 
if len(files)%LoopFiles != 0:
    loops += 1

st = ti()

Moves = []

for k in range(loops):
    if k == loops -1:
        tfiles = BeforeTamping[k*LoopFiles:]
    else:
        tfiles = BeforeTamping[k*LoopFiles:(k+1)*LoopFiles]
    #Results = Parallel(n_jobs=LoopFiles)(delayed(DeviationVelocity)(file) for file in tfiles)
    Results = Parallel(n_jobs=LoopFiles)(delayed(SmoothMoves)(file) for file in tfiles)
    #Results =[]
    #for file in tfiles:
    #    Results.append(DeviationVelocity(file))
    #    print(file, (ti()-st)/60.0)
    for result in Results:
        Moves.append(result)
    print(k, (ti()-st)/60.0)
        
Sequences = []
Outputs = []
for move in Moves:
    Seq, Out = split_sequences(move.T,TimeSteps,StepSize)
    Sequences.append(Seq)
    Outputs.append(Out)
MoveSegments = []
for seq in Sequences:
    for mv in seq:
        MoveSegments.append(mv)
NextDataPoint = []
for out in Outputs:
    for pt in out:
        NextDataPoint.append(np.reshape(pt,(1,3)))